## Deliverable 2. Create a Customer Travel Destinations Map.

In [55]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,55.98,81,1,7.85,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,67.32,77,100,13.67,light rain
2,2,Longyearbyen,SJ,78.2186,15.6401,51.64,76,75,14.97,broken clouds
3,3,Saint-Georges,GD,12.0564,-61.7485,80.28,83,75,9.22,broken clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,68.56,98,73,8.01,light rain


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Saint-Georges,GD,12.0564,-61.7485,80.28,83,75,9.22,broken clouds
5,5,Hilo,US,19.7297,-155.0900,76.28,95,100,3.00,moderate rain
6,6,Ternate,ID,0.8000,127.4000,79.09,85,100,11.59,overcast clouds
12,12,Lixourion,GR,38.2019,20.4314,84.20,39,0,11.50,clear sky
13,13,Alofi,NU,-19.0595,-169.9187,75.09,78,69,8.05,broken clouds
15,15,Butaritari,KI,3.0707,172.7902,82.44,80,44,17.07,light rain
16,16,Awjilah,LY,29.1081,21.2869,77.09,35,0,10.42,clear sky
17,17,Gizo,SB,-8.1030,156.8419,83.25,71,100,13.47,overcast clouds
21,21,Vaini,TO,-21.2000,-175.2000,77.16,94,40,9.22,scattered clouds
24,24,Mormugao,IN,15.4000,73.8000,80.71,84,100,21.81,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                193
City                   193
Country                193
Lat                    193
Lng                    193
Max Temp               193
Humidity               193
Cloudiness             193
Wind Speed             193
Current Description    193
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Saint-Georges,GD,80.28,broken clouds,12.0564,-61.7485,
5,Hilo,US,76.28,moderate rain,19.7297,-155.0900,
6,Ternate,ID,79.09,overcast clouds,0.8000,127.4000,
12,Lixourion,GR,84.20,clear sky,38.2019,20.4314,
13,Alofi,NU,75.09,broken clouds,-19.0595,-169.9187,
15,Butaritari,KI,82.44,light rain,3.0707,172.7902,
16,Awjilah,LY,77.09,clear sky,29.1081,21.2869,
17,Gizo,SB,83.25,overcast clouds,-8.1030,156.8419,
21,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
24,Mormugao,IN,80.71,overcast clouds,15.4000,73.8000,


In [56]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")  
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [61]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   176
Country                176
Max Temp               176
Current Description    176
Lat                    176
Lng                    176
Hotel Name             176
dtype: int64

In [62]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [63]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [64]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))